# Building Final Data Structure

In this notebook, I will build my final data structure for the selected athletes filteres from the athletes_overview CSV file using the riding frequency and duration active criteria. First, I will re-run my hr_max and hr_min calculations for the athletes that were filtered out for having an extremely high hr_max using the median based function. I will then select those athletes with plausible HR readings. 

In [1]:
# Directory change
# RUN ONCE PER SESSION
import os

os.chdir(r"..")

## 1. Re-running HR min and max script

In [2]:
# Imports
import polars as pl
from joblib import Parallel, delayed
from src.data import Athlete


In [3]:
# Loading list of athletes selected initially
df_females = pl.read_csv(r"data\interim\df_females.csv")
df_males = pl.read_csv(r"data\interim\df_males.csv")

df_athletes = pl.concat([df_females, df_males])

In [4]:
# Defining processor function for parallel processing of HR min and max
def processor(row):
    # Get athlete id from dataframe and create an athlete object
    athlete = Athlete(row["id"])

    # Calculate HR min and max for athlete
    athlete.get_hr_min_max()

    # Add hr_min and hr_max to row and create a new polars dataframe
    row["max_hr"] = athlete.max_hr
    row["min_hr"] = athlete.min_hr

    output_df = pl.DataFrame(row)

    return output_df

In [5]:
# Getting min and max HRs for all athletes

output_list = Parallel(n_jobs=-1)(
    delayed(processor)(row) for row in df_athletes.iter_rows(named=True)
)

In [6]:
df_athletes = pl.concat(output_list)

In [12]:
df_athletes.sort(by="max_hr", descending=True).head(10)

id,gender,yob,numberOfRides,duration,rideFrequency,max_hr,min_hr
str,str,i64,i64,i64,f64,f64,f64
"""09407f03-8a99-499c-9830-5c657b…","""F""",1995,121,240,0.504167,null,null
"""17c30fad-f9f1-452f-afea-16c811…","""M""",1997,342,539,0.634508,null,null
"""74114d44-6c7f-4b11-a9c5-ba33f2…","""M""",1967,872,1732,0.503464,214.0,62.0
"""a1c8b721-1490-4b9f-8bae-65694e…","""M""",1965,921,1696,0.543042,204.95,43.15242
"""3785483e-f809-45e7-8dcf-95cceb…","""F""",2002,638,1023,0.623656,203.0,67.0
"""5ae846a2-fb61-4553-a41f-99db55…","""M""",2000,461,448,1.029018,203.0,42.0
"""5d9be2e9-fe18-46b0-83d3-ff3c4a…","""M""",1973,276,375,0.736,203.0,52.0
"""a0631b3a-c8cb-4e7f-949e-d102c6…","""F""",1997,1562,2184,0.715201,202.0,56.0
"""f3771ea1-af65-4b36-90ad-6ffb5a…","""F""",2002,743,1298,0.572419,201.0,91.0


In [8]:
df_athletes.write_csv(r"data\processed\df_athletes_hrs_updated.csv")

## 2. Calculating HRR, MMP, and TRIMP
I will calculate HRR, MMP, and TRIMP for all athletes in the above dataframe.

In [2]:
# Imports
import polars as pl
from joblib import Parallel, delayed
from src.data import Athlete

In [3]:
df = pl.read_csv(r"data\processed\df_athletes_hrs_updated.csv")

# Filter out athletes with no heart rate data
df = df.filter(
    pl.col("max_hr").is_not_null(),
    pl.col("min_hr").is_not_null(),
)

In [4]:
# Defining processor function for parallel processing
def processor(row):
    # Get athlete id from dataframe and create an athlete object
    athlete = Athlete(row["id"])
    athlete.max_hr = row["max_hr"]
    athlete.min_hr = row["min_hr"]

    # Calculating HRR, MMP, and TRIMP
    df_hrr = athlete.process_hrr()
    df_mmp = athlete.process_mmp(0.85, 4)
    df_trimp = athlete.process_trimp()

    # Returning output
    return [df_hrr, df_mmp, df_trimp]

In [5]:
# Processing all activities in parallel

output_list = Parallel(n_jobs=-1)(
    delayed(processor)(row) for row in df.iter_rows(named=True)
)

In [6]:
hrr_list = [athlete[0] for athlete in output_list]
mmp_list = [athlete[1] for athlete in output_list]
trimp_list = [athlete[2] for athlete in output_list]

In [7]:
df_hrr = pl.concat(hrr_list)
df_mmp = pl.concat(mmp_list)
df_trimp = pl.concat(trimp_list)

In [8]:
# Saving dataframes to interim folder
df_hrr.write_csv(r"data\interim\df_hrr.csv")
df_mmp.write_csv(r"data\interim\df_mmp.csv")
df_trimp.write_csv(r"data\interim\df_trimp.csv")

In [ ]:
# Chaining the join operations to merge the dataframes
df_hrr_mmp = (
    df_hrr.join(df_mmp, on=["athlete_id", "gender", "week_no"], how="full")
    .with_columns(
        pl.col("athlete_id").fill_null(pl.col("athlete_id_right")),
        pl.col("gender").fill_null(pl.col("gender_right")),
        pl.col("week_no").fill_null(pl.col("week_no_right")),
    )
    .drop(["athlete_id_right", "gender_right", "week_no_right"])
)

df_merged = (
    df_hrr_mmp.join(df_trimp, on=["athlete_id", "gender", "week_no"], how="full")
    .with_columns(
        pl.col("athlete_id").fill_null(pl.col("athlete_id_right")),
        pl.col("gender").fill_null(pl.col("gender_right")),
        pl.col("week_no").fill_null(pl.col("week_no_right")),
    )
    .drop(["athlete_id_right", "gender_right", "week_no_right"])
)

In [56]:
df_merged.describe()

statistic,athlete_id,gender,week_no,activity_id,date,hrr_window_start_secs,hrr_window_end_secs,HRR(30),activity_id_right,date_right,mmp_window_start_secs,mmp_window_end_secs,maximal_mean_power,total_weekly_trimp
str,str,str,f64,str,str,f64,f64,f64,str,str,f64,f64,f64,f64
"""count""","""44430""","""44430""",44430.0,"""30361""","""30361""",30361.0,30361.0,30361.0,"""40911""","""40911""",40911.0,40911.0,40911.0,44430.0
"""null_count""","""0""","""0""",0.0,"""14069""","""14069""",14069.0,14069.0,14069.0,"""3519""","""3519""",3519.0,3519.0,3519.0,0.0
"""mean""",null,null,87.993248,null,"""2018-05-02 01:48:27.446592""",6804.885511,6833.885511,27.229736,null,"""2018-04-16 23:54:18.923859""",5352.093691,5592.093691,227.9396,778.016799
"""std""",null,null,76.085869,null,null,6398.007095,6398.007095,13.355365,null,null,5791.957865,5791.957865,59.964235,352.006195
"""min""","""01843f3a-8883-4a76-a60f-223c92…","""F""",-95.0,"""2011_08_23_17_36_12.csv""","""2011-08-23 15:36:12""",11.0,40.0,0.0,"""2011_03_09_17_14_02.csv""","""2011-03-09 16:14:02""",0.0,240.0,0.0,0.030264
"""25%""",null,null,30.0,null,"""2017-08-23 14:12:19""",2514.0,2543.0,19.0,null,"""2017-07-30 07:29:48""",1602.0,1842.0,190.745833,527.553196
"""50%""",null,null,69.0,null,"""2018-06-17 14:06:43""",4973.0,5002.0,27.0,null,"""2018-06-06 14:49:14""",3501.0,3741.0,225.4,743.576099
"""75%""",null,null,119.0,null,"""2019-05-02 03:26:20""",9129.0,9158.0,35.0,null,"""2019-04-20 03:36:15""",7121.0,7361.0,267.158333,1011.716497
"""max""","""fe4bcccf-399f-4cd4-b385-e3b053…","""M""",463.0,"""2020_05_01_16_02_10.csv""","""2020-05-01 22:02:10""",101935.0,101964.0,147.0,"""2020_05_05_19_28_16.csv""","""2020-05-05 18:28:16""",128539.0,128779.0,597.591667,4737.612364


In [57]:
# Removing duplicate rows
df_merged = df_merged.with_columns(
    pl.col("activity_id").alias("activity_id_hrr"),
    pl.col("date").alias("date_hrr"),
    pl.col("activity_id_right").alias("activity_id_mmp"),
    pl.col("date_right").alias("date_mmp"),
    pl.col("maximal_mean_power").alias("MMP"),
).select(
    [
        "athlete_id",
        "gender",
        "week_no",
        "activity_id_hrr",
        "date_hrr",
        "hrr_window_start_secs",
        "hrr_window_end_secs",
        "HRR(30)",
        "activity_id_mmp",
        "date_mmp",
        "mmp_window_start_secs",
        "mmp_window_end_secs",
        "MMP",
        "total_weekly_trimp",
    ]
)

In [58]:
df_merged

athlete_id,gender,week_no,activity_id_hrr,date_hrr,hrr_window_start_secs,hrr_window_end_secs,HRR(30),activity_id_mmp,date_mmp,mmp_window_start_secs,mmp_window_end_secs,MMP,total_weekly_trimp
str,str,i64,str,datetime[μs],i64,i64,i64,str,datetime[μs],i64,i64,f64,f64
"""01843f3a-8883-4a76-a60f-223c92…","""F""",45,null,null,null,null,null,"""2018_05_18_13_18_17.csv""",2018-05-18 20:18:17,572,812,204.208333,590.130813
"""01843f3a-8883-4a76-a60f-223c92…","""F""",45,null,null,null,null,null,"""2018_05_18_13_32_00.csv""",2018-05-18 20:32:00,7,247,277.9,590.130813
"""01843f3a-8883-4a76-a60f-223c92…","""F""",45,null,null,null,null,null,"""2018_05_20_08_08_27.csv""",2018-05-20 14:08:27,9224,9464,232.133333,590.130813
"""01843f3a-8883-4a76-a60f-223c92…","""F""",46,null,null,null,null,null,"""2018_05_23_11_18_03.csv""",2018-05-23 17:18:03,3370,3610,245.025,494.650524
"""01843f3a-8883-4a76-a60f-223c92…","""F""",46,null,null,null,null,null,"""2018_05_23_11_18_03.csv""",2018-05-23 17:18:03,7065,7305,209.279167,494.650524
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""d9e4e8a4-2a85-474d-867d-4d5f25…","""M""",124,null,null,null,null,null,null,null,null,null,null,689.977117
"""06f81197-504a-478b-a347-052d4d…","""F""",183,null,null,null,null,null,null,null,null,null,null,833.658667
"""091ff782-d063-4411-9f40-9b679e…","""M""",3,null,null,null,null,null,null,null,null,null,null,520.216961


In [59]:
# Saving df_merged to interim. This dataset has multiple rows for each week.

df_merged.write_csv(r"data\interim\df_merged.csv")

In [60]:
# Grouping by athlete_id and week and selecting the median and 75th percentile of HRR and the maximum MMP for each week
df_merged = (
    df_merged.group_by(["athlete_id", "gender", "week_no"])
    .agg(
        pl.col("HRR(30)").median().alias("50_HRR(30)"),
        pl.col("HRR(30)").quantile(0.75).alias("75_HRR(30)"),
        pl.col("MMP").max().alias("MMP_max"),
        pl.col("total_weekly_trimp").first().alias("TRIMP"),
    )
    .sort(["athlete_id", "week_no"])
)

In [61]:
df_merged

athlete_id,gender,week_no,50_HRR(30),75_HRR(30),MMP_max,TRIMP
str,str,i64,f64,f64,f64,f64
"""01843f3a-8883-4a76-a60f-223c92…","""F""",-1,null,null,null,29.393435
"""01843f3a-8883-4a76-a60f-223c92…","""F""",1,null,null,null,372.633562
"""01843f3a-8883-4a76-a60f-223c92…","""F""",2,null,null,null,99.6454
"""01843f3a-8883-4a76-a60f-223c92…","""F""",3,null,null,null,217.230806
"""01843f3a-8883-4a76-a60f-223c92…","""F""",4,null,null,null,267.000822
…,…,…,…,…,…,…
"""fe4bcccf-399f-4cd4-b385-e3b053…","""M""",64,null,null,null,364.959667
"""fe4bcccf-399f-4cd4-b385-e3b053…","""M""",65,null,null,223.554167,458.038017
"""fe4bcccf-399f-4cd4-b385-e3b053…","""M""",66,null,null,240.033333,395.231501


In [63]:
# This dataset has one row per week per athlete, with the median and 75th percentile of HRR and the maximum MMP for each week.

df_merged.write_csv(r"data\processed\final_df.csv")